<a href="https://colab.research.google.com/github/lrsilsbee/Machine-Learning/blob/main/Lauren_Silsbee_evaluating_performance_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the houseprices data from Thinkful's database.
Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
Do you think your model is satisfactory? If so, why?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houseprices_df = pd.read_sql_query('select * from houseprices',con=engine)
engine.dispose()

In [ ]:
#create dummies first
houseprices_df = pd.concat([houseprices_df,pd.get_dummies(houseprices_df.centralair, prefix="centralair", drop_first=True)], axis=1)
houseprices_df = pd.concat([houseprices_df,pd.get_dummies(houseprices_df.street, prefix="street", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(houseprices_df.centralair, prefix="centralair", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(houseprices_df.street, prefix="street", drop_first=True).columns)

#build model

Y = houseprices_df['saleprice']
X = houseprices_df[['garagecars', 'grlivarea', 'overallqual'] + dummy_column_names]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     833.2
Date:                Tue, 10 Nov 2020   Prob (F-statistic):               0.00
Time:                        14:16:44   Log-Likelihood:                -17557.
No. Observations:                1460   AIC:                         3.513e+04
Df Residuals:                    1454   BIC:                         3.516e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.086e+05   1.71e+04     -6.364      0.000   -1.42e+05   -7.51e+04
garagecars     2.06e+04   1817.061     11.338      0.000     1.7e+04    2.42e+04
grlivarea       51.6233      2.558     20.181      0.000      46.605      56.641
overallqual   2.635e+04   1091.545     24.139      0.000    2.42e+04    2.85e+04
centralair_Y  7937.6163   2258.425      3.515      0.000    3507.496    1.24e+04
centralair_Y  7937.6163   2258.425      3.515      0.000    3507.496    1.24e+04
street_Pave   -358.1762   8334.331     -0.043      0.966   -1.67e+04     1.6e+04
street_Pave   -358.1762   8334.331     -0.043      0.966   -1.67e+04     1.6e+04
==============================================================================
Omnibus:                      438.984   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10184.406
Skew:                           0.845   Prob(JB):                         0.00
Kurtosis:                      15.828   Cond. No.                     2.00e+19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.42e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

The Adj. R Squared (.74) and R Squared (.741) indicate that this model explains 74% of the variance in Y. This is decent (and certainly better than the weather model's explanatory model) but can be improved. The F statistic is significant (p < .05), which tell us that this model explains more variance in Y than a reduced model does. The AIC is 3.513e+04 and the BIC is 3.516e+04.

In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables.

In [ ]:
#Review correlations

numeric_columns = houseprices_df.select_dtypes(['int64', 'float64']).columns
np.abs(houseprices_df[numeric_columns].iloc[:,1:].corr().loc[:,"saleprice"]).sort_values(ascending=False)

saleprice        1.000000
overallqual      0.790982
grlivarea        0.708624
garagecars       0.640409
garagearea       0.623431
totalbsmtsf      0.613581
firstflrsf       0.605852
fullbath         0.560664
totrmsabvgrd     0.533723
yearbuilt        0.522897
yearremodadd     0.507101
garageyrblt      0.486362
masvnrarea       0.477493
fireplaces       0.466929
bsmtfinsf1       0.386420
lotfrontage      0.351799
wooddecksf       0.324413
secondflrsf      0.319334
openporchsf      0.315856
halfbath         0.284108
lotarea          0.263843
bsmtfullbath     0.227122
bsmtunfsf        0.214479
bedroomabvgr     0.168213
kitchenabvgr     0.135907
enclosedporch    0.128578
screenporch      0.111447
poolarea         0.092404
mssubclass       0.084284
overallcond      0.077856
mosold           0.046432
threessnporch    0.044584
yrsold           0.028923
lowqualfinsf     0.025606
miscval          0.021190
bsmthalfbath     0.016844
bsmtfinsf2       0.011378
Name: saleprice, dtype: float64

totalbsmtsf is correlated with the target. Let's try including that variable into the model.

In [ ]:
Y = houseprices_df['saleprice']
X = houseprices_df[['garagecars', 'grlivarea', 'overallqual', 'totalbsmtsf'] + dummy_column_names]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     774.9
Date:                Tue, 10 Nov 2020   Prob (F-statistic):               0.00
Time:                        14:17:52   Log-Likelihood:                -17496.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1453   BIC:                         3.504e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.116e+05   1.64e+04     -6.819      0.000   -1.44e+05   -7.95e+04
garagecars    1.816e+04   1757.247     10.333      0.000    1.47e+04    2.16e+04
grlivarea       46.4221      2.498     18.583      0.000      41.522      51.322
overallqual   2.305e+04   1087.838     21.193      0.000    2.09e+04    2.52e+04
totalbsmtsf     31.9073      2.844     11.221      0.000      26.329      37.485
centralair_Y  5883.7888   2174.967      2.705      0.007    1617.377    1.02e+04
centralair_Y  5883.7888   2174.967      2.705      0.007    1617.377    1.02e+04
street_Pave   2391.0829   8001.622      0.299      0.765   -1.33e+04    1.81e+04
street_Pave   2391.0829   8001.622      0.299      0.765   -1.33e+04    1.81e+04
==============================================================================
Omnibus:                      416.222   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            37343.690
Skew:                          -0.215   Prob(JB):                         0.00
Kurtosis:                      27.773   Cond. No.                     1.03e+21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.15e-33. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Adding the basement's square footage increased the R squared and Adj. R squared slightly. It also slightly decreased the AIC and BIC. Let's add another variable to see if we can improve the model even more.

In [ ]:
Y = houseprices_df['saleprice']
X = houseprices_df[['garagecars', 'grlivarea', 'overallqual', 'totalbsmtsf', 'garagearea'] + dummy_column_names]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     665.3
Date:                Tue, 10 Nov 2020   Prob (F-statistic):               0.00
Time:                        14:18:30   Log-Likelihood:                -17495.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1452   BIC:                         3.505e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.131e+05   1.64e+04     -6.901      0.000   -1.45e+05    -8.1e+04
garagecars     1.43e+04   3018.710      4.738      0.000    8380.649    2.02e+04
grlivarea       46.1095      2.505     18.408      0.000      41.196      51.023
overallqual   2.312e+04   1088.023     21.247      0.000     2.1e+04    2.53e+04
totalbsmtsf     30.9514      2.907     10.649      0.000      25.250      36.653
garagearea      16.4658     10.485      1.570      0.117      -4.101      37.032
centralair_Y  5745.6082   2175.651      2.641      0.008    1477.854       1e+04
centralair_Y  5745.6082   2175.651      2.641      0.008    1477.854       1e+04
street_Pave   3325.4743   8019.689      0.415      0.678   -1.24e+04    1.91e+04
street_Pave   3325.4743   8019.689      0.415      0.678   -1.24e+04    1.91e+04
==============================================================================
Omnibus:                      430.668   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            41062.877
Skew:                          -0.279   Prob(JB):                         0.00
Kurtosis:                      28.975   Cond. No.                     5.62e+20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.85e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

The above model hardly changed the goodness of fit indicators (probably bc its so closely related to some variables already in the model). The second and third models, based on the goodness of fit indicators, perform only slightly better than the first model. 76% of explained variance is satisfactor, but I'm sure this model could still be improved. Perhaps including interaction terms or taking the log of certain variables could improve the Adj. R squared and AIC/BIC estimates.